In [3]:
%reload_ext autoreload
%autoreload 2

import cv2
import numpy as np
import os
import os.path as osp
from calibrate_table import calibrate_table_by_aruco, calibrate_table_by_markers
from table_markers import get_table_markers_coords_in_table_frame_by_aruco
from params import table_aruco_size, box_aruco_size, box_size, K, D, target_table_markers

In [ ]:
view = "top"
folders = ["data/test_boxes_distance_aruco/", "data/test_boxes_distance_segm/"]
out_folder = None

image_files = [list(map(lambda im_file: osp.join(folder, im_file), sorted(os.listdir(folder))))
                    for folder in folders]
image_files = sum(image_files, [])

table_frames = list()
for i, image_file in enumerate(image_files):
    image = cv2.imread(image_file)
    table_frame, _ = calibrate_table_by_aruco(image, view, K, D, table_aruco_size)
    assert table_frame is not None
    table_frames.append(table_frame)
    
    if out_folder is not None:
        camera2table = table_frame.origin2plane()
        rvec, _ = cv2.Rodrigues(camera2table[0:3, 0:3])
        tvec = camera2table[0:3, 3]
        draw = image.copy()
        cv2.drawFrameAxes(draw, K, D, rvec, tvec, 0.1)
        cv2.imwrite(osp.join(out_folder, f"{i}.jpg"), draw)

t_diffs = list()
r_diffs = list()
n = len(table_frames)
for i in range(n - 1):
    for j in range(i + 1, n):
        table_frame_i = table_frames[i].origin2plane()
        table_frame_j = table_frames[j].origin2plane()
        diff = np.matmul(np.linalg.inv(table_frame_i), table_frame_j)
        t_diff = diff[:3, 3]
        r_diff = cv2.Rodrigues(diff[:3, :3])[0].squeeze()
        t_diffs.append(t_diff)
        r_diffs.append(r_diff)
t_diffs = np.array(t_diffs)
r_diffs = np.array(r_diffs)

max_t_diff = np.max(np.linalg.norm(t_diffs, axis=-1))
max_r_diff = np.max(np.linalg.norm(r_diffs, axis=-1))
print(f"Max t diff: {max_t_diff * 100} cm")
print(f"Max r diff: {max_r_diff * 180 / np.pi} deg")
print()

max_t_diff_xy = np.max(np.linalg.norm(t_diffs[:, :2], axis=-1))
max_t_diff_z = np.max(t_diffs[:, 2])
max_r_diff_z = np.max(r_diffs[:, 2])
print(f"Max t diff xy: {max_t_diff_xy * 100} cm")
print(f"Max t diff z: {max_t_diff_z * 100} cm")
print(f"Max r diff z: {max_r_diff_z * 180 / np.pi} deg")